# Proyecto 3: Conversión y calibración de archivos raw EK80 con echopype

Emilio Mayorga, https://github.com/emiliom/, 2023-3-2

https://github.com/Intercoonecta/proy3-acustica

In [1]:
from pathlib import Path

import numpy as np
import xarray as xr

import echopype as ep

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
rawdirpath = Path("/home/jovyan/shared/proyecto3")

In [3]:
# Esto lo usamos para pruebas nada mas
# raw_nombres = ['ebes202210-D20221011-T012820.raw', 'ebes202210-D20221011-T012021.raw']

De https://github.com/Intercoonecta/proy3-acustica/issues/3

gain = 18.36; sa corr = 0.02; EQA = -12.5. Todo esto solo para la frecuencia 38 kHz.

----------------

Información extraída de uno de los archivos convertidos:

```python
ed["Sonar/Beam_group1"].channel.data, ed["Sonar/Beam_group1"].frequency_nominal.data

(array(['WBT Mini 262036-7 ES38-18|200-18C',
        'WBT Mini 262036-8 ES38-18|200-18C'], dtype='<U33'),
 array([ 38000., 200000.]))
```

In [4]:
def convertir_y_calibrar(raw_ruta):
    # convertir
    ed = ep.open_raw(raw_ruta, sonar_model='EK80')
    
    # modificar parametros de calibracion para 38kHz solamente
    # Nota: el argumento cal_params de compute_Sv no funciona actualmente
    # https://github.com/OSOceanAcoustics/echopype/issues/950
    # https://github.com/OSOceanAcoustics/echopype/issues/887
    # Entonces, modificamos esos parpameteros aquí
    channel_38k = str(ed["Vendor_specific"].channel[0].data)
    # La frecuencia 38k correponde al primer canal, channel[0]
    # (Aunque idealmente debería haber seleccionado, por cada archivo,
    # primero frequency_nominal == 3800 y luego el canal que corresponde a esa frequencia
    ed["Vendor_specific"].gain_correction.loc[dict(channel=channel_38k)] = 18.36
    ed["Vendor_specific"].sa_correction.loc[dict(channel=channel_38k)] = 0.02
    ed["Sonar/Beam_group1"].equivalent_beam_angle.loc[dict(channel=channel_38k)] = -12.5
    
    # calibrar
    Sv_ds = ep.calibrate.compute_Sv(
        ed, 
        waveform_mode='CW',
        encode_mode='complex',
        env_params = {
            'temperature': 26.15, # en Centígrados
            'salinity': 34.96,    # en PSU
            'pressure': 50,       # en dbar (tomado del grupo Environment)
        },
        # cal_params=cal_params
    )
    
    return Sv_ds

## Procesar todos los archivos raw en el directorio.

Generar archivos netcdf de Sv (calibrados) y guardarlos en `shared-readwrite/proyecto3/calibrado`

In [5]:
for raw_ruta in rawdirpath.glob('*.raw'):
    print(f"Procesando {raw_ruta.name}")
    Sv_ds = convertir_y_calibrar(raw_ruta)
    
    calibrado_ruta = Path("/home/jovyan/shared-readwrite/proyecto3/calibrado") / f"{raw_ruta.stem}.calibrado.nc"
    Sv_ds.to_netcdf(calibrado_ruta)

Procesando ebes202210-D20221011-T012021.raw
Procesando ebes202210-D20221011-T012301.raw
Procesando ebes202210-D20221011-T012542.raw
Procesando ebes202210-D20221011-T012820.raw
Procesando ebes202210-D20221011-T013100.raw
Procesando ebes202210-D20221011-T013341.raw
Procesando ebes202210-D20221011-T013619.raw
Procesando ebes202210-D20221011-T013856.raw
Procesando ebes202210-D20221011-T014137.raw
Procesando ebes202210-D20221011-T014430.raw
Procesando ebes202210-D20221011-T014705.raw
Procesando ebes202210-D20221011-T014946.raw
Procesando ebes202210-D20221011-T015226.raw
Procesando ebes202210-D20221011-T015500.raw
Procesando ebes202210-D20221011-T015741.raw


Abrir uno de los archivos netcdf de Sv que acabamos de crear, para confirmar que se abre bien

In [6]:
ds = xr.open_dataset(Path("/home/jovyan/shared-readwrite/proyecto3/calibrado") / "ebes202210-D20221011-T012820.calibrado.nc")

In [7]:
ds

<xarray.Dataset>
Dimensions:                (channel: 2, range_sample: 22150, ping_time: 160,
                            filenames: 1, time3: 1)
Coordinates:
  * channel                (channel) object 'WBT Mini 262036-7 ES38-18|200-18...
  * range_sample           (range_sample) int64 0 1 2 3 ... 22147 22148 22149
  * ping_time              (ping_time) datetime64[ns] 2022-10-11T01:28:20.293...
  * time3                  (time3) datetime64[ns] 2022-10-11T01:28:20.293000192
Dimensions without coordinates: filenames
Data variables: (12/13)
    Sv                     (channel, ping_time, range_sample) float64 ...
    echo_range             (channel, ping_time, range_sample) float64 ...
    frequency_nominal      (channel) float64 ...
    temperature            float64 ...
    salinity               float64 ...
    pressure               int64 ...
    ...                     ...
    sound_absorption       (channel) float64 ...
    sa_correction          (ping_time, channel) float64 ...
    gain_correction        (ping_time, channel) float64 ...
    equivalent_beam_angle  (channel, ping_time) float64 ...
    source_filenames       (filenames) object ...
    water_level            (time3) float64 ...
Attributes:
    processing_software_name:     echopype
    processing_software_version:  0.6.0
    processing_time:              2023-03-02T19:26:32Z
    processing_function:          calibrate.compute_Sv